In [6]:
import datetime
from glob import glob
import math
import matplotlib.pyplot as plt
import nibabel as nb
from nilearn import image
import numpy as np
import os
import pandas as pd
import random
import tensorflow as tf
import timeit
import warnings
%matplotlib inline

In [7]:
import os
import warnings

import tensorflow as tf
import src.models.basic_cnn as model
from src.data_io import InputFnFactory, _get_data
import datetime

tf.logging.set_verbosity(tf.logging.INFO)

if __name__ == '__main__':
    log_dir = "logs"
    current_run_subdir = os.path.join(
        "run_" + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    model_dir = os.path.join(log_dir, model.name, "106x128x110")#current_run_subdir)

    run_config = tf.estimator.RunConfig(model_dir=model_dir)

    params = tf.contrib.training.HParams(
        target_shape=(106, 128, 110),
        model_dir=model_dir
    )

    ds = InputFnFactory(target_shape=params.target_shape,
                        n_epochs=100,
                        train_src_folder="/home/smantra/finalproject/data/",
                        train_cache_prefix="/home/smantra/finalproject/cache_train_hires/",
                        eval_src_folder="/home/smantra/finalproject/eval/",
                        eval_cache_prefix="/home/smantra/finalproject/cache_eval_hires/",
                        batch_size=4
                        )

    # Workaround for cache iterator concurency issues. Iterate over the whole
    # training dataset without counterbalancing to make sure everything is
    # preprocessed and cached
    if not os.path.exists(ds.train_cache_prefix + ".index"):
        print("Preprocessing the training set")
        with tf.Session() as sess:
            train_dataset = _get_data(batch_size=ds.batch_size,
                                      src_folder=ds.train_src_folder,
                                      n_epochs=2,
                                      cache_prefix=ds.train_cache_prefix,
                                      shuffle=False,
                                      target_shape=params.target_shape,
                                      balance_dataset=True)

            train_dataset = train_dataset.make_one_shot_iterator()
            while True:
                try:
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        features, labels = sess.run(train_dataset.get_next())
                except tf.errors.OutOfRangeError:
                    break
        print("Finished preprocessing the training set")

    train_spec = tf.estimator.TrainSpec(input_fn=ds.train_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=ds.eval_input_fn,
                                      steps=None,
                                      start_delay_secs=0,
                                      throttle_secs=1200)

    estimator = tf.estimator.Estimator(model_fn=model.model_fn,
                                       params=params,
                                       config=run_config)

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Preprocessing the training set
<TensorSliceDataset shapes: ((),), types: (tf.string,)>


UnknownError: ImageFileError: Cannot work out file type of "/home/smantra/finalproject/data/sub-0050049_T1w.nii.gz"
Traceback (most recent call last):

  File "/home/shared/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 157, in __call__
    ret = func(*args)

  File "/home/smantra/finalproject/src/data_io.py", line 43, in _read_and_resample
    nii = nb.load(path_str)

  File "/home/shared/anaconda3/lib/python3.6/site-packages/nibabel/loadsave.py", line 49, in load
    filename)

nibabel.filebasedimages.ImageFileError: Cannot work out file type of "/home/smantra/finalproject/data/sub-0050049_T1w.nii.gz"


	 [[Node: read_and_resample_0 = PyFunc[Tin=[DT_STRING, DT_DOUBLE, DT_INT32], Tout=[DT_FLOAT], token="pyfunc_6"](arg0, read_and_resample_placeholder, read_and_resample_placeholder_1)]]
	 [[Node: IteratorGetNext_127 = IteratorGetNext[output_shapes=[[?,106,128,110], [?]], output_types=[DT_FLOAT, DT_INT32], _device="/job:localhost/replica:0/task:0/device: